In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
#from fastai.imports import *
#from fastai.torch_imports import *
#from fastai.dataset import *

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50,edgeitems=20)

#from sklearn_pandas import DataFrameMapper
#from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler
#import operator

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH = '../../../data/rossmann-structured/'

## CLEANING AND FEATURE ENGINEERING

In [4]:
os.listdir(PATH)

['store.csv',
 'state_names.csv',
 'df',
 'googletrend.csv',
 'weather.csv',
 'store_states.csv',
 'train.csv',
 'test.csv',
 'archive']

In [5]:
csv_names = ['store.csv',
 'state_names.csv',
 'googletrend.csv',
 'weather.csv',
 'store_states.csv',
 'train.csv',
 'test.csv']

data_frames_table = []

In [6]:
for name in csv_names:
    data_frames_table.append(pd.read_csv(f'{PATH}{name}',low_memory=False))

In [7]:
len(data_frames_table)

7

In [8]:
for df in data_frames_table:
    display(df.head())

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [9]:
store,state_names,googletrend,weather,store_states,train,test=data_frames_table

In [10]:
train.StateHoliday = train.StateHoliday!='0'
train.SchoolHoliday = train.SchoolHoliday!=0
test.StateHoliday = test.StateHoliday!='0'
test.SchoolHoliday = test.SchoolHoliday!=0

In [11]:
def join_df(df_left,df_right,left_on,right_on=None,how='left',suffix='_X'):
    if right_on==None: right_on=left_on
    return pd.merge(left=df_left,right=df_right,left_on=left_on,
                    right_on=right_on,how=how,suffixes=('',suffix))

In [12]:
store_states = join_df(store_states,state_names,left_on='State')

In [13]:
joined_df = join_df(train,store_states,left_on='Store')
joined_df_test = join_df(test,store_states,left_on='Store')

In [14]:
len(joined_df[joined_df['StateName'].isna()]),len(joined_df_test[joined_df_test['StateName'].isna()])

(0, 0)

In [15]:
joined_df = join_df(joined_df,store,left_on='Store')
joined_df_test = join_df(joined_df_test,store,left_on='Store')

In [16]:
len(joined_df[joined_df['StoreType'].isna()]),len(joined_df_test[joined_df_test['StoreType'].isna()])

(0, 0)

In [17]:
joined_df = join_df(joined_df,weather,left_on=['Date','StateName'],right_on=['Date','file'])
joined_df_test = join_df(joined_df_test,weather,left_on=['Date','StateName'],right_on=['Date','file'])

In [18]:
len(joined_df[joined_df['Max_TemperatureC'].isna()]),len(joined_df_test[joined_df_test['Max_TemperatureC'].isna()])

(0, 0)

In [19]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]

In [20]:
googletrend['State'] = googletrend['file'].apply(lambda file: file.split('_')[-1])

In [21]:
googletrend.loc[googletrend['State']=='NI','State']='HB,NI'

In [22]:
googletrend.head()

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN


In [23]:
add_datepart(joined_df, "Date", drop=False)
add_datepart(joined_df_test, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)

In [ ]:
joined_df = join_df(joined_df, googletrend, ["State","Year","Week"])
joined_df_test = join_df(joined_df_test, googletrend, ["State","Year","Week"])

In [ ]:
len(joined_df[joined_df.trend.isnull()]),len(joined_df_test[joined_df_test.trend.isnull()])

(0, 0)

In [ ]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [ ]:
joined_df = join_df(joined_df, trend_de[['trend','Year','Week']], ["Year", "Week"], suffix='_DE')
joined_df_test = join_df(joined_df_test, trend_de[['trend','Year','Week']], ["Year", "Week"], suffix='_DE')

In [ ]:
len(joined_df[joined_df.trend_DE.isnull()]),len(joined_df_test[joined_df_test.trend_DE.isnull()])

(0, 0)

In [ ]:
joined_df.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'State', 'StateName', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'file', 'Max_TemperatureC',
       'Mean_TemperatureC', 'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC',
       'Min_DewpointC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h',
       'Max_Gust_SpeedKm_h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 

In [ ]:
for df in (joined_df,joined_df_test):
    for column in df.columns:
        if column.endswith('_X'):
            df.drop(column,axis=1,inplace=True)

### Missing Values processing

In [ ]:
joined_df.isnull().values.any(),joined_df_test.isnull().values.any()

(True, True)

In [ ]:
joined_df.columns[joined_df.isna().any()].tolist(),joined_df_test.columns[joined_df_test.isna().any()].tolist()

(['CompetitionDistance',
  'CompetitionOpenSinceMonth',
  'CompetitionOpenSinceYear',
  'Promo2SinceWeek',
  'Promo2SinceYear',
  'PromoInterval',
  'Max_VisibilityKm',
  'Mean_VisibilityKm',
  'Min_VisibilitykM',
  'Max_Gust_SpeedKm_h',
  'CloudCover',
  'Events'],
 ['Open',
  'CompetitionDistance',
  'CompetitionOpenSinceMonth',
  'CompetitionOpenSinceYear',
  'Promo2SinceWeek',
  'Promo2SinceYear',
  'PromoInterval',
  'Max_VisibilityKm',
  'Mean_VisibilityKm',
  'Min_VisibilitykM',
  'Max_Gust_SpeedKm_h',
  'CloudCover',
  'Events'])

In [ ]:
for df in (joined_df,joined_df_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [ ]:
joined_df.columns[joined_df.isna().any()].tolist(),joined_df_test.columns[joined_df_test.isna().any()].tolist()

(['CompetitionDistance',
  'PromoInterval',
  'Max_VisibilityKm',
  'Mean_VisibilityKm',
  'Min_VisibilitykM',
  'Max_Gust_SpeedKm_h',
  'CloudCover',
  'Events'],
 ['Open',
  'CompetitionDistance',
  'PromoInterval',
  'Max_VisibilityKm',
  'Mean_VisibilityKm',
  'Min_VisibilitykM',
  'Max_Gust_SpeedKm_h',
  'CloudCover',
  'Events'])

In [ ]:
for df in (joined_df,joined_df_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [ ]:
for df in (joined_df,joined_df_test):
    df.loc[df["CompetitionDaysOpen"]<0,'CompetitionDaysOpen']=0
    df.loc[df.CompetitionOpenSinceYear==1900,'CompetitionDaysOpen']=0

In [ ]:
for df in (joined_df,joined_df_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined_df.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5])

In [ ]:
for df in (joined_df,joined_df_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [ ]:
for df in (joined_df,joined_df_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [ ]:
joined_df.to_feather(f'{PATH}/df/joined_df')
joined_df_test.to_feather(f'{PATH}/df/joined_df_test')

### Durations
- days past state&school holiday, promo
- days to state&school holiday, promo
- running number of days of state&school holiday, promo in (b) week BEFORE
- running number of days of state&school holiday, promo in (f) week FOLLOWING

In [ ]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [ ]:
#df = train[columns]
df = train[columns].append(test[columns])

In [ ]:
df.Date = df['Date'].apply(pd.to_datetime)

In [ ]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [ ]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
df = df.set_index("Date")

In [ ]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

In [ ]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()


In [ ]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [ ]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [ ]:
df.drop(columns,1,inplace=True)

In [ ]:
df.to_feather(f'{PATH}/df/df_working')

In [ ]:
df = pd.read_feather(f'{PATH}/df/df_working')

In [ ]:
df["Date"] = pd.to_datetime(df.Date)

In [ ]:
df.columns.tolist()

In [ ]:
joined_df = join_df(joined_df, df, ['Store', 'Date'])
joined_df_test = join_df(joined_df_test, df, ['Store', 'Date'])

In [ ]:
joined_df.reset_index(inplace=True)
joined_df_test.reset_index(inplace=True)

In [ ]:
joined_df.to_feather(f'{PATH}/df/joined_df')
joined_df_test.to_feather(f'{PATH}/df/joined_df_test')

## FEATURE ENCODING/CREATION

In [ ]:
joined_df=pd.read_feather(f'{PATH}/df/joined_df')
joined_df_test=pd.read_feather(f'{PATH}/df/joined_df_test')

In [ ]:
joined_df.head().T.head(100)